In [92]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce

In [93]:
dtypes = {'id': 'int32', 
'titulo': 'object', 
'descripcion': 'object', 
'tipodepropiedad': 'category', 
'direccion': 'object', 
'ciudad': 'object', 
'provincia': 'category', 
'antiguedad': 'float', 
'habitaciones': 'float', 
'garages': 'float', 
'banos': 'float', 
'metroscubiertos': 'float', 
'metrostotales': 'float', 
'idzona': 'object', 
'lat': 'float64', 
'lng': 'float64', 
'gimnasio': 'float16', 
'usosmultiples': 'float16', 
'piscina': 'float16', 
'escuelascercanas': 'float16', 
'centroscomercialescercanos': 'float16', 
'precio': 'float32', }

data = pd.read_csv("train.csv", dtype = dtypes)
data.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


# Pre-procesamiento de data

In [94]:
data["ciudad"].nunique()

875

In [95]:
data.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [96]:
data.isnull().sum().sum()

535127

In [97]:
data.size

5520000

In [98]:
(data.isnull().sum().sum()/(data.size))*100

9.694329710144928

Los Nulls representan el 10% de los datos.

# Levanto el csv de test para calcularle los features en paralelo

In [99]:
test = pd.read_csv("test.csv", dtype = dtypes)
test.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,NaN,...,NaN,NaN,19.408668,-99.246767,2013-07-20 00:00:00,0.0,0.0,0.0,0.0,0.0
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,NaN,1.0,1.0,...,67.0,113851.0,21.032480,-89.592424,2015-10-24 00:00:00,0.0,0.0,0.0,0.0,0.0
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,100.0,23620.0,19.332829,-99.152913,2015-05-30 00:00:00,0.0,0.0,0.0,0.0,1.0
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,86.0,129347.0,16.860487,-99.878383,2015-04-02 00:00:00,0.0,0.0,0.0,0.0,0.0
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,76.0,57125.0,19.640482,-99.127273,2013-08-15 00:00:00,0.0,0.0,0.0,1.0,1.0


# Se completan NULLs con extensión de valores válidos

Lo que se hace es ordenar el dataset por tipodepropiedad. A continuacion se llama a la funcion fillna con el metodo ffill, que propaga la ultima observacion valida hasta la nueva observacion valida. De esta manera deberiamos llenar de forma mas o menos correcta. En caso de que haya quedado algun null, utilizo bfill

## Ordeno el dataset

In [100]:
data = data.sort_values(by = 'tipodepropiedad')

In [101]:
test = test.sort_values(by = 'tipodepropiedad')

## Metroscubiertos

In [102]:
data[['metroscubiertos']] = data[['metroscubiertos']].fillna(method = "ffill")

In [103]:
data["metroscubiertos"].isnull().sum().sum()

0

In [104]:
test[['metroscubiertos']] = test[['metroscubiertos']].fillna(method = "ffill")

In [105]:
test["metroscubiertos"].isnull().sum().sum()

1

In [106]:
test['metroscubiertos'] = test['metroscubiertos'].fillna(method = "bfill")

In [107]:
test["metroscubiertos"].isnull().sum().sum()

0

## Metrostotales

In [108]:
data['metrostotales'] = data['metrostotales'].fillna(method = "ffill")

In [109]:
data['metrostotales'].isnull().sum()

0

In [110]:
test['metrostotales'] = test['metrostotales'].fillna(method = "ffill")

In [111]:
test['metrostotales'].isnull().sum()

0

## Antiguedad

In [112]:
data['antiguedad'] = data['antiguedad'].fillna(method = "ffill")

In [113]:
data["antiguedad"].isnull().sum().sum()

14

In [114]:
data['antiguedad'] = data['antiguedad'].fillna(method = "bfill")

In [115]:
data["antiguedad"].isnull().sum().sum()

0

In [116]:
test['antiguedad'] = test['antiguedad'].fillna(method = "ffill")

In [117]:
test["antiguedad"].isnull().sum().sum()

0

## Banos

In [118]:
data['banos'] = data['banos'].fillna(method = "ffill")

In [119]:
data["banos"].isnull().sum().sum()

0

In [120]:
test['banos'] = test['banos'].fillna(method = "ffill")

In [121]:
test["banos"].isnull().sum().sum()

1

In [122]:
test['banos'] = test['banos'].fillna(method = "bfill")

In [123]:
test["banos"].isnull().sum().sum()

0

## Garages

In [124]:
data['garages'] = data['garages'].fillna(method = "ffill")

In [125]:
data["garages"].isnull().sum().sum()

0

In [126]:
test['garages'] = test['garages'].fillna(method = "ffill")

In [127]:
test["garages"].isnull().sum().sum()

3

In [128]:
test['garages'] = test['garages'].fillna(method = "bfill")

In [129]:
test["garages"].isnull().sum().sum()

0

## Re ordeno el dataset segun ciudad para fillear latitud y longitud (que al parecer es muy importante)

In [130]:
data = data.sort_values(by = 'ciudad')

In [131]:
test = test.sort_values(by = 'ciudad')

## Latitud

In [132]:
data['lat'] = data['lat'].fillna(method = "ffill")

In [133]:
data["lat"].isnull().sum().sum()

2

In [134]:
data['lat'] = data['lat'].fillna(method = "bfill")

In [135]:
data["lat"].isnull().sum().sum()

0

In [136]:
test['lat'] = test['lat'].fillna(method = "ffill")

In [137]:
test["lat"].isnull().sum().sum()

0

## Longitud

In [138]:
data['lng'] = data['lng'].fillna(method = "ffill")

In [139]:
data["lng"].isnull().sum().sum()

2

In [140]:
data['lng'] = data['lng'].fillna(method = "bfill")

In [141]:
data["lng"].isnull().sum().sum()

0

In [142]:
test['lng'] = test['lng'].fillna(method = "ffill")

In [143]:
test["lng"].isnull().sum().sum()

0

# Preparacion del set de datos de entrenamiento (features)

La idea es preparar el set con los datos para exportar y que despues el modelo simplemente levante, separe en set de entrenamiento y test, entrene y devuelva una prediccion.

Se procede a calcular features. Cada feature se agregara al DataFrame final que tendra que levantar despues el modelo. Ojo que dentro de este DataFrame tambien va a estar el precio (que es el label).

In [144]:
train_set = pd.DataFrame()

In [145]:
test_set = pd.DataFrame()

Primero de todo, se agrega el id y el precio asocidado. A partir de esto, se agregaran los features que se consideren necesarios.

In [146]:
train_set["id"] = data["id"]
train_set["precio"] = data["precio"]

In [147]:
train_set.head()

,id,precio
11188,192202,315000.0
52476,94838,2500000.0
30789,7819,1315600.0
13057,62385,8500000.0
138118,284662,680000.0


A partir de esto se generan features.

Igual pero para test_set. Solo que sin el precio.

In [148]:
test_set["id"] = test["id"]

In [149]:
test_set.head()

,id
58125,47104
14468,175113
14055,253170
4218,222487
56787,45818


# Se codifican las variables categoricas

In [150]:
data['train'] = True
test['train'] = False
combined = pd.concat([data, test], sort = False)

In [151]:
train_set['train'] = True
test_set["train"] = False
combined_set = pd.concat([train_set, test_set], sort = False)

In [152]:
combined['tipodepropiedad'].nunique()

24

In [153]:
combined['ciudad'].nunique()

921

Ciudad tiene demasiados valores posibles para OneHotEncoding

In [154]:
combined['provincia'].nunique()

32

In [155]:
combined_set['tipodepropiedad'] = combined['tipodepropiedad']
combined_set['provincia'] = combined['provincia']

In [156]:
var_categoricas = ['tipodepropiedad', 'provincia']

In [157]:
one_hot_enc = ce.OneHotEncoder(handle_unknown = 'ignore')
one_hot_encoded = one_hot_enc.fit_transform(combined_set[var_categoricas])
one_hot_encoded.columns

Index(['tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'provincia_20', 'provincia_21', 'provincia_22', 'provincia_23',
  

In [158]:
combined_set = combined_set.join(one_hot_encoded)
combined_set.columns

Index(['id', 'precio', 'train', 'tipodepropiedad', 'provincia',
       'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'p

In [159]:
train_set = combined_set[combined_set["train"] == True]
test_set = combined_set[combined_set["train"] == False]

In [160]:
train_set = train_set.drop(columns = ['tipodepropiedad', 'provincia' , 'train'])

In [161]:
train_set.columns

Index(['id', 'precio', 'tipodepropiedad_1', 'tipodepropiedad_2',
       'tipodepropiedad_3', 'tipodepropiedad_4', 'tipodepropiedad_5',
       'tipodepropiedad_6', 'tipodepropiedad_7', 'tipodepropiedad_8',
       'tipodepropiedad_9', 'tipodepropiedad_10', 'tipodepropiedad_11',
       'tipodepropiedad_12', 'tipodepropiedad_13', 'tipodepropiedad_14',
       'tipodepropiedad_15', 'tipodepropiedad_16', 'tipodepropiedad_17',
       'tipodepropiedad_18', 'tipodepropiedad_19', 'tipodepropiedad_20',
       'tipodepropiedad_21', 'tipodepropiedad_22', 'tipodepropiedad_23',
       'tipodepropiedad_24', 'tipodepropiedad_25', 'provincia_1',
       'provincia_2', 'provincia_3', 'provincia_4', 'provincia_5',
       'provincia_6', 'provincia_7', 'provincia_8', 'provincia_9',
       'provincia_10', 'provincia_11', 'provincia_12', 'provincia_13',
       'provincia_14', 'provincia_15', 'provincia_16', 'provincia_17',
       'provincia_18', 'provincia_19', 'provincia_20', 'provincia_21',
       'provincia_

In [162]:
test_set = test_set.drop(columns = ['tipodepropiedad', 'precio' , 'provincia', 'train'])

In [163]:
test_set.columns

Index(['id', 'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'provincia_20', 'provincia_21', 'provincia_22', 'provincia_2

## Se agregan todas las columnas numericas que ya se tienen

In [164]:
data.dtypes

id                               int32
titulo                          object
descripcion                     object
tipodepropiedad               category
direccion                       object
ciudad                          object
provincia                     category
antiguedad                     float64
habitaciones                   float64
garages                        float64
banos                          float64
metroscubiertos                float64
metrostotales                  float64
idzona                          object
lat                            float64
lng                            float64
fecha                           object
gimnasio                       float16
usosmultiples                  float16
piscina                        float16
escuelascercanas               float16
centroscomercialescercanos     float16
precio                         float32
train                             bool
dtype: object

Agrego los valores numericos que no poseen NULLs.

In [165]:
data_num = data.loc[:,["id", "usosmultiples", "piscina", "escuelascercanas",
                      "centroscomercialescercanos"]]

In [166]:
train_set = train_set.merge(data_num, on = ["id"], how = "inner")
train_set.head()

,id,precio,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,...,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,254099,2273000.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,254099,2273000.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,53461,3600000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,1.0
3,53461,3600000.0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0.0,0.0,1.0,1.0
4,247984,1200000.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [167]:
test_num = test.loc[:,["id", "usosmultiples", "piscina", "escuelascercanas",
                      "centroscomercialescercanos"]]

In [168]:
test_set = test_set.merge(test_num, on = ["id"], how = "inner")
test_set.head()

,id,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,tipodepropiedad_9,...,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,4941,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,51775,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,51775,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0.0,0.0,0.0,0.0
4,115253,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0


## Se agregan las variables que fueron filleadas

In [169]:
data_num_fill = data.loc[:, ["id", "antiguedad", "metrostotales", "metroscubiertos", 'garages', 'banos', 'lat', 'lng']]

In [170]:
train_set = train_set.merge(data_num_fill, on = 'id', how = 'inner')
train_set.head()

,id,precio,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,...,piscina,escuelascercanas,centroscomercialescercanos,antiguedad,metrostotales,metroscubiertos,garages,banos,lat,lng
0,254099,2273000.0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,20.0,80.0,80.0,1.0,2.0,19.393583,-99.159777
1,254099,2273000.0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,20.0,80.0,80.0,1.0,2.0,19.393583,-99.159777
2,53461,3600000.0,0,0,0,0,0,1,0,0,...,0.0,1.0,1.0,10.0,180.0,268.0,2.0,2.0,19.310205,-99.227655
3,53461,3600000.0,0,0,0,1,0,0,0,0,...,0.0,1.0,1.0,10.0,180.0,268.0,2.0,2.0,19.310205,-99.227655
4,247984,1200000.0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,5.0,166.0,144.0,2.0,2.0,20.662590,-103.212280


In [171]:
test_num_fill = test.loc[:, ["id", "antiguedad", "metrostotales", "metroscubiertos", 'garages', 'banos', 'lat', 'lng']]

In [172]:
test_set = test_set.merge(test_num_fill, on = 'id', how = 'inner')
test_set.head()

,id,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,tipodepropiedad_9,...,piscina,escuelascercanas,centroscomercialescercanos,antiguedad,metrostotales,metroscubiertos,garages,banos,lat,lng
0,4941,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,29.0,112.0,300.0,1.0,4.0,19.408668,-99.246767
1,4941,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,29.0,112.0,300.0,1.0,4.0,19.408668,-99.246767
2,51775,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,67.0,67.0,1.0,1.0,21.032480,-89.592424
3,51775,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,67.0,67.0,1.0,1.0,21.032480,-89.592424
4,115253,0,1,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,100.0,87.0,1.0,2.0,19.332829,-99.152913


# Nuevos features

# Promedio de metroscubiertos por tipo de propiedad

In [173]:
metros_mean = data.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "mean"}).reset_index()

In [174]:
metros_mean.head()

,tipodepropiedad,metroscubiertos
0,Apartamento,117.049615
1,Bodega comercial,250.651494
2,Casa,194.067338
3,Casa en condominio,187.819091
4,Casa uso de suelo,253.124294


In [175]:
metros_mean.rename(columns = {"metroscubiertos" : "metroscubiertos_mean_tipodeprop"}, inplace = True)
metros_mean.head()

,tipodepropiedad,metroscubiertos_mean_tipodeprop
0,Apartamento,117.049615
1,Bodega comercial,250.651494
2,Casa,194.067338
3,Casa en condominio,187.819091
4,Casa uso de suelo,253.124294


In [176]:
data = data.merge(metros_mean, on = 'tipodepropiedad')

In [177]:
train_set = train_set.merge(data[['id', 'metroscubiertos_mean_tipodeprop']], on = "id")

### Lo mismo pero para test

In [178]:
metros_mean = test.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "mean"}).reset_index()

In [179]:
metros_mean.head()

,tipodepropiedad,metroscubiertos
0,Apartamento,116.177500
1,Bodega comercial,250.569444
2,Casa,193.996470
3,Casa en condominio,186.668296
4,Casa uso de suelo,242.400901


In [180]:
metros_mean.rename(columns = {"metroscubiertos" : "metroscubiertos_mean_tipodeprop"}, inplace = True)
metros_mean.head()

,tipodepropiedad,metroscubiertos_mean_tipodeprop
0,Apartamento,116.177500
1,Bodega comercial,250.569444
2,Casa,193.996470
3,Casa en condominio,186.668296
4,Casa uso de suelo,242.400901


In [181]:
test = test.merge(metros_mean, on = 'tipodepropiedad')

In [182]:
test_set = test_set.merge(test[['id', 'metroscubiertos_mean_tipodeprop']], on = "id")

# Varianza de metroscubiertos por tipo de propiedad

In [183]:
metros_var = data.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "var"}).reset_index()

In [184]:
metros_var.rename(columns = {"metroscubiertos" : "metroscubiertos_var_tipodeprop"}, inplace = True)
metros_var.fillna(method = "ffill", inplace = True)

In [185]:
data = data.merge(metros_var, on = 'tipodepropiedad')

In [186]:
train_set = train_set.merge(data[['id', 'metroscubiertos_var_tipodeprop']], on = "id")

### Lo mismo pero para test

In [187]:
metros_var = test.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "var"}).reset_index()

In [188]:
metros_var.rename(columns = {"metroscubiertos" : "metroscubiertos_var_tipodeprop"}, inplace = True)
metros_var.fillna(method = "ffill", inplace = True)

In [189]:
test = test.merge(metros_var, on = 'tipodepropiedad')

In [190]:
test_set = test_set.merge(test[['id', 'metroscubiertos_var_tipodeprop']], on = "id")

# Se exporta el train_set final para uso de los modelos

## Suma de nulls totales. TIENE que ser 0. Sino el modelo no corre

In [191]:
train_set.isnull().sum().sum()

0

In [192]:
train_set.to_csv("train_set.csv")

In [193]:
test_set.isnull().sum().sum()

0

In [194]:
test_set.to_csv("test_set.csv")